# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [114]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        #K(x,xi) = 1 + sum(x * xi)^d
        d=3
        kernel_sum = np.dot(data_set, data_set.T) ** d
        kernel = 1+ kernel_sum

    return kernel

In [115]:
from sklearn.datasets import load_iris

iris = load_iris()
data_set = iris.data

build_kernel(data_set, kernel_type='poly')

array([[ 65257.129576,  52693.198749,  50777.309927, ..., 135243.687637,
        136910.392264, 111216.247129],
       [ 52693.198749,  42912.760501,  41028.927849, ..., 114296.958477,
        114792.256   ,  93639.512421],
       [ 50777.309927,  41028.927849,  39513.447416, ..., 105891.949891,
        107172.875   ,  87057.109568],
       ...,
       [135243.687637, 114296.958477, 105891.949891, ..., 557239.592989,
        575516.133432, 464945.021723],
       [136910.392264, 114792.256   , 107172.875   , ..., 575516.133432,
        602281.721125, 482998.531736],
       [111216.247129,  93639.512421,  87057.109568, ..., 464945.021723,
        482998.531736, 389978.008616]])

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [116]:
def choose_set_for_label(data_set, label):
    
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.15, random_state=20)
    
    train_labels[train_labels<1] = -1
    test_labels[test_labels<1] = -1

    return train_data_set, test_data_set, train_labels, test_labels

In [117]:
def get_labels_count(data_set):
    return len(data_set)

Use the code that we have implemented earlier:

In [118]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [119]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt

In [120]:
iris = load_iris()
data_set = iris.data
labels = iris.target
# modify this part 
data_set = data_set[labels!=2]
labels = labels[labels!=2]

train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels)
objects_count = get_labels_count(train_data_set)

lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  9.7172e+01 -1.2899e+03  2e+03  2e-01  2e-15
 1:  6.1193e+01 -1.2247e+02  2e+02  5e-03  2e-15
 2:  7.3497e+00 -1.6917e+01  2e+01  3e-16  2e-15
 3: -5.0158e-01 -3.8438e+00  3e+00  2e-16  9e-16
 4: -1.1816e+00 -1.8190e+00  6e-01  2e-16  3e-16
 5: -1.4235e+00 -1.6558e+00  2e-01  3e-16  2e-16
 6: -1.4772e+00 -1.5711e+00  9e-02  2e-16  2e-16
 7: -1.5050e+00 -1.5164e+00  1e-02  2e-16  2e-16
 8: -1.5096e+00 -1.5100e+00  4e-04  2e-16  2e-16
 9: -1.5098e+00 -1.5098e+00  4e-06  3e-16  2e-16
10: -1.5098e+00 -1.5098e+00  4e-08  2e-16  2e-16
Optimal solution found.
0.8666666666666667
